In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from databricks.connect import DatabricksSession
spark = DatabricksSession.builder.getOrCreate()

In [ ]:
from dishes_forecasting.run_features import Args
args = Args(company="GL", env="dev")

In [ ]:
from dishes_forecasting.logger_config import logger
from dishes_forecasting.features.build_features import *
from dishes_forecasting.inputs.get_data import *
from constants.companies import get_company_by_code
from dishes_forecasting.utils import read_yaml
from dishes_forecasting.paths import CONFIG_DIR

company_code = args.company
logger.info(f"Running feature generation step for {company_code}...")
company = get_company_by_code(company_code=company_code)
company_id = company.company_id
logger.info("Fetching configs for ...")
input_configs = read_yaml(directory=CONFIG_DIR, filename="inputs.yml")

input_config = input_configs[company_code]

feature_configs = read_yaml(directory=CONFIG_DIR, filename="features.yml")

logger.info("Fetching data...")
(
    df_flex_orders,
    df_weekly_variations,
    df_recipes,
    df_recipe_ingredients,
    df_recipe_price_ratings,
) = get_input_data(company_id=company_id, input_config=input_config, env=args.env)

logger.info("Creating feature dataframes...")
df_weekly_dishes_features, df_recipe_features = build_features(
    df_flex_orders=df_flex_orders,
    df_weekly_variations=df_weekly_variations,
    df_recipes=df_recipes,
    df_recipe_ingredients=df_recipe_ingredients,
    df_recipe_price_ratings=df_recipe_price_ratings,
    feature_configs=feature_configs,
    language=company.language,
)

In [ ]:
for i in df_recipe_features.columns:
    print(i)

In [ ]:
from dishes_forecasting.run_features import run_features, Args

args = Args(company="GL", env="dev")
df_weekly_dishes_features, df_recipe_features = run_features(args=args)


In [ ]:
df_weekly_dishes_features.head()

In [ ]:
# Check if the table exists
table_name = "weekly_dishes_features"
if spark.catalog.tableExists(table_name):
    print(f"The table {table_name} exists.")
else:
    print(f"The table does not exist")